In [1]:
import numpy as np
import os
path = 'data/data_20_800.npy'
path = 'data/data_20_800_new_.npy'
#path = 'data/data_20_800_normal_solved.npy'
path = 'data/data_100_200.npy'
#path = 'data/data_200_200.npy'
#path = 'data/data_500_200.npy'
with open(path, 'rb') as f:
    data = np.load(f, allow_pickle=True)
len(data)

200

In [2]:
def calculate_tardiness(schedule, processing_times, due_dates):
    current_time = 0
    tardiness = 0
    for job in schedule:
        current_time += processing_times[job]
        tardiness += max(0, current_time - due_dates[job])
    return tardiness

def final_tardiness(p,d,seq):
    p = np.array(p)
    d = np.array(d)

    tardiness = np.cumsum(p[seq])-d[seq]
    tardiness = tardiness[tardiness>0]
    
    total_tardiness = tardiness.sum()

    return total_tardiness

In [3]:
def EDD(p, d):
    assigned_jobs = np.argsort(d)
    return assigned_jobs

In [4]:
def MDD_Challenger(processing_times: np.ndarray, due_dates: np.ndarray) -> np.ndarray:
    """ Find mathematical heuristic function skeleton for the single machine scheduling problem. Each job is assigned to the machine exactly once. Do not manipulate original due dates or processing times.

    Args:
        processing_times: A numpy array representing processing times of jobs.
        due_dates: A numpy array representing due dates of jobs.

    Return:
        A numpy array representing indices of jobs assigned in order as the result of applying the mathematical function to the inputs.
    """
    """Ultimate version of the job assignment function for the single machine scheduling problem."""
    current_time = 0
    schedule = []
    unscheduled_jobs = np.arange(len(processing_times))
    
    sort_order = np.lexsort((processing_times[unscheduled_jobs], due_dates[unscheduled_jobs]))
    unscheduled_jobs = unscheduled_jobs[sort_order]
    
    while unscheduled_jobs.size > 0:
        urgencies = np.maximum(processing_times[unscheduled_jobs] * 1.1 + current_time, due_dates[unscheduled_jobs])
        urgency_multiplier = (processing_times[unscheduled_jobs] / (current_time + np.max(processing_times[unscheduled_jobs])))
        urgency_multiplier[urgency_multiplier > 1] = 1
        urgency_bonus = (urgency_multiplier ** 2) / (1 + urgency_multiplier ** 2)
        urgencies *= (1 + urgency_bonus)
        impact_on_current_time = processing_times[unscheduled_jobs] / (current_time + np.sum(processing_times[unscheduled_jobs]) / len(unscheduled_jobs))
        urgencies += impact_on_current_time
        next_job = unscheduled_jobs[np.argmin(urgencies)]
        schedule.append(next_job)
        current_time += processing_times[next_job]
        unscheduled_jobs = np.delete(unscheduled_jobs, np.where(unscheduled_jobs == next_job))

        #search
        '''best_tardiness = calculate_tardiness(schedule, p, d)
        best_schedule = schedule
        for i in range(len(schedule) - 1):
            trial_schedule = schedule[:i] + schedule[i+1:] + [schedule[i]]
            trial_tardiness = calculate_tardiness(trial_schedule, p, d)
            if trial_tardiness < best_tardiness:
                best_tardiness = trial_tardiness
                best_schedule = trial_schedule[:]
        schedule = best_schedule'''
    
    return np.array(schedule)

In [5]:
def Augmented_MDD_Challenger(processing_times: np.ndarray, due_dates: np.ndarray) -> np.ndarray:
    """ Find mathematical heuristic function skeleton for the single machine scheduling problem. Each job is assigned to the machine exactly once. Do not manipulate original due dates or processing times.

    Args:
        processing_times: A numpy array representing processing times of jobs.
        due_dates: A numpy array representing due dates of jobs.

    Return:
        A numpy array representing indices of jobs assigned in order as the result of applying the mathematical function to the inputs.
    """
    """Ultimate version of the job assignment function for the single machine scheduling problem."""
    current_time = 0
    schedule = []
    unscheduled_jobs = np.arange(len(processing_times))
    
    sort_order = np.lexsort((processing_times[unscheduled_jobs], due_dates[unscheduled_jobs]))
    unscheduled_jobs = unscheduled_jobs[sort_order]
    
    while unscheduled_jobs.size > 0:
        urgencies = np.maximum(processing_times[unscheduled_jobs] * 1.1 + current_time, due_dates[unscheduled_jobs])
        urgency_multiplier = (processing_times[unscheduled_jobs] / (current_time + np.max(processing_times[unscheduled_jobs])))
        urgency_multiplier[urgency_multiplier > 1] = 1
        urgency_bonus = (urgency_multiplier ** 2) / (1 + urgency_multiplier ** 2)
        urgencies *= (1 + urgency_bonus)
        impact_on_current_time = processing_times[unscheduled_jobs] / (current_time + np.sum(processing_times[unscheduled_jobs]) / len(unscheduled_jobs))
        urgencies += impact_on_current_time
        next_job = unscheduled_jobs[np.argmin(urgencies)]
        schedule.append(next_job)
        current_time += processing_times[next_job]
        unscheduled_jobs = np.delete(unscheduled_jobs, np.where(unscheduled_jobs == next_job))

        #search
        best_tardiness = calculate_tardiness(schedule, p, d)
        best_schedule = schedule
        for i in range(len(schedule) - 1):
            trial_schedule = schedule[:i] + schedule[i+1:] + [schedule[i]]
            trial_tardiness = calculate_tardiness(trial_schedule, p, d)
            if trial_tardiness < best_tardiness:
                best_tardiness = trial_tardiness
                best_schedule = trial_schedule[:]
        schedule = best_schedule
    
    return np.array(schedule)

In [6]:
#yedek
def EDD_Challenger(processing_times: np.ndarray, due_dates: np.ndarray) -> np.ndarray:
    """ Find mathematical heuristic function skeleton for the single machine scheduling problem. Each job is assigned to the machine exactly once. Do not change the original due dates and processing times of jobs.

    Args:
        processing_times: A numpy array representing processing times of jobs.
        due_dates: A numpy array representing due dates of jobs.

    Return:
        A numpy array representing indices of jobs assigned in order as the result of applying the mathematical function to the inputs.
    """
    """Improved version of `assignment_v1` that considers physical relationships of inputs and aims for concise code."""
    if len(processing_times) != len(due_dates):
        raise ValueError("The lengths of processing_times and due_dates must be equal.")
    
    assigned_jobs = np.lexsort((due_dates, processing_times), axis=0)
    
    for i in range(1, len(assigned_jobs)):
        j = i
        print(processing_times, due_dates)
        while j > 0 and due_dates[assigned_jobs[j]] < due_dates[assigned_jobs[j - 1]]:
            non_zero_idx = np.nonzero(processing_times[assigned_jobs[:j]])[0]
            if non_zero_idx.size > 0:
                #print(non_zero_idx)
                available_time = sum(processing_times[assigned_jobs][non_zero_idx])
            else:
                #print("asf")
                available_time = 0
    
            if due_dates[assigned_jobs[j - 1]] > available_time:
                assigned_jobs[j], assigned_jobs[j - 1] = assigned_jobs[j - 1], assigned_jobs[j]
            j -= 1
    
    return assigned_jobs

In [7]:
def EDD_Challenger(processing_times: np.ndarray, due_dates: np.ndarray) -> np.ndarray:
    """ Find mathematical heuristic function skeleton for the single machine scheduling problem. Each job is assigned to the machine exactly once. Do not change the original due dates and processing times of jobs.

    Args:
        processing_times: A numpy array representing processing times of jobs.
        due_dates: A numpy array representing due dates of jobs.

    Return:
        A numpy array representing indices of jobs assigned in order as the result of applying the mathematical function to the inputs.
    """
    """Improved version of `assignment_v1` that considers physical relationships of inputs and aims for concise code."""
    if len(processing_times) != len(due_dates):
        raise ValueError("The lengths of processing_times and due_dates must be equal.")
    
    S = np.lexsort((due_dates, processing_times), axis=0)
    
    for i in range(1, len(S)):
        j = i
        while j > 0 and due_dates[S[j]] < due_dates[S[j - 1]]:
            non_zero_idx = np.nonzero(processing_times[S[:j]])[0]
            if non_zero_idx.size > 0:
                
                available_time = sum(processing_times[S][non_zero_idx])
            else:
                available_time = 0
    
            if due_dates[S[j - 1]] > available_time:
                S[j], S[j - 1] = S[j - 1], S[j]
            j -= 1
    
    print(S)
    return S

In [8]:
k = 0
d,p = data[k]['due_dates'], data[k]['processing_times']
S = np.lexsort((d, p), axis=0)
i,j = 15,15
d[S[j]],d[S[j-1]]
p[S[:j]]
np.nonzero(S[:j])[0]
EDD_Challenger(d, p)


[55 66 30 64 37 18 93 86 16 33 76 45 34 36 92 59 40 38 88 29 65 22 39  7
 97 74 47 32 54 90 43 23 46  4 52 84  9  1 79 28 57 61 27 56 25  8 48  6
 15 94 83 13 96  2 69 67 91 31 50 49 12 14 20 68 26 82 10 98  0 80 71 89
 62 63  5 53 95 21 87 81 85 78  3 11 73 77 60 58 44 99 42 72 19 41 17 24
 35 70 75 51]


array([55, 66, 30, 64, 37, 18, 93, 86, 16, 33, 76, 45, 34, 36, 92, 59, 40,
       38, 88, 29, 65, 22, 39,  7, 97, 74, 47, 32, 54, 90, 43, 23, 46,  4,
       52, 84,  9,  1, 79, 28, 57, 61, 27, 56, 25,  8, 48,  6, 15, 94, 83,
       13, 96,  2, 69, 67, 91, 31, 50, 49, 12, 14, 20, 68, 26, 82, 10, 98,
        0, 80, 71, 89, 62, 63,  5, 53, 95, 21, 87, 81, 85, 78,  3, 11, 73,
       77, 60, 58, 44, 99, 42, 72, 19, 41, 17, 24, 35, 70, 75, 51])

In [9]:
def EDD_Challenger2(processing_times: np.ndarray, due_dates: np.ndarray) -> np.ndarray:
    """ Find mathematical heuristic function skeleton for the single machine scheduling problem. Each job is assigned to the machine exactly once. Do not change the original due dates and processing times of jobs.

    Args:
        processing_times: A numpy array representing processing times of jobs.
        due_dates: A numpy array representing due dates of jobs.

    Return:
        A numpy array representing indices of jobs assigned in order as the result of applying the mathematical function to the inputs.
    """
    perturbation_factor = 0.1 * (np.max(due_dates) - np.min(due_dates))
    modifier = np.maximum(due_dates - processing_times, 0) + perturbation_factor * np.abs(np.random.normal(loc=1, scale=0.1, size=len(due_dates)))
    weighted_flowtimes = ((modifier + perturbation_factor) * np.exp(-processing_times / np.max(processing_times)) + processing_times * 10) / np.sum(modifier + perturbation_factor)
    weighted_flowtimes[np.isclose(weighted_flowtimes, 0)] = 1e-7
    return np.argsort(weighted_flowtimes)

In [10]:
def MDD(p, d):
    # Initialize current time
    current_time = 0

    # Create an array to track the scheduling order
    schedule = []

    # Number of jobs
    num_jobs = len(p)

    # Create an index array for tracking unscheduled jobs
    unscheduled_jobs = np.arange(num_jobs)

    # While there are unscheduled jobs
    while unscheduled_jobs.size > 0:
        # Calculate urgencies for each unscheduled job
        #urgencies = np.maximum(p[unscheduled_jobs], d[unscheduled_jobs] - current_time)
        urgencies = np.maximum(p[unscheduled_jobs]+ current_time, d[unscheduled_jobs] )
        # Find the job with the minimum urgency
        index_min_urgency = np.argmin(urgencies)
        job_to_schedule = unscheduled_jobs[index_min_urgency]
        
        # Append job id (index + 1 for 1-based id) to schedule
        schedule.append(job_to_schedule)
        
        # Update current time
        current_time += p[job_to_schedule]
        
        # Remove the job from the list of unscheduled jobs
        unscheduled_jobs = np.delete(unscheduled_jobs, index_min_urgency)
    return schedule

In [11]:
def AUG_MDD(p, d):
    # Initialize current time
    current_time = 0

    # Create an array to track the scheduling order
    schedule = []

    # Number of jobs
    num_jobs = len(p)

    # Create an index array for tracking unscheduled jobs
    unscheduled_jobs = np.arange(num_jobs)

    # While there are unscheduled jobs
    while unscheduled_jobs.size > 0:
        # Calculate urgencies for each unscheduled job
        #urgencies = np.maximum(p[unscheduled_jobs], d[unscheduled_jobs] - current_time)
        urgencies = np.maximum(p[unscheduled_jobs]+ current_time, d[unscheduled_jobs] )
        # Find the job with the minimum urgency
        index_min_urgency = np.argmin(urgencies)
        job_to_schedule = unscheduled_jobs[index_min_urgency]
        
        # Append job id (index + 1 for 1-based id) to schedule
        schedule.append(job_to_schedule)
        
        # Update current time
        current_time += p[job_to_schedule]
        
        # Remove the job from the list of unscheduled jobs
        unscheduled_jobs = np.delete(unscheduled_jobs, index_min_urgency)

        best_tardiness = calculate_tardiness(schedule, p, d)
        best_schedule = schedule
        for i in range(len(schedule) - 1):
            trial_schedule = schedule[:i] + schedule[i+1:] + [schedule[i]]
            trial_tardiness = calculate_tardiness(trial_schedule, p, d)
            if trial_tardiness < best_tardiness:
                best_tardiness = trial_tardiness
                best_schedule = trial_schedule[:]
        schedule = best_schedule
    return schedule

In [12]:
def pannerselvam(processing_times, due_dates):
    num_jobs = len(processing_times)
    # Step 1: Initialization
    index = [0] * num_jobs  # Tracks whether a job is scheduled (1) or not (0)
    final_sequence = []  # The final sequence of jobs
    L = 0  # Position in the final sequence

    while L < num_jobs:
        # Step 2: Calculate the sum of processing times of unscheduled jobs
        SUMT = sum(processing_times[i] for i in range(num_jobs) if index[i] == 0)

        # Step 3 & 4: Calculate slack and slack per unit processing time for unscheduled jobs
        USi = []
        for i in range(num_jobs):
            if index[i] == 0:
                Si = SUMT - due_dates[i]
                USi.append((Si / processing_times[i], i))

        # Step 5: Find the job with the maximum USi
        max_USi, job_k = max(USi)  # Get the job with maximum USi

        # Step 6 & 7: Update the final sequence and mark job K as scheduled
        L += 1
        final_sequence.append(job_k)
        index[job_k] = 1

    # Step 9: Calculate completion times and tardiness
    completion_times = []
    total_tardiness = 0
    current_time = 0
    for job in final_sequence:
        current_time += processing_times[job]
        completion_times.append(current_time)
        tardiness = max(0, completion_times[-1] - due_dates[job])
        total_tardiness += tardiness

    # printing
    #print("Final sequence:", [job + 1 for job in final_sequence])  # Adjusting for 1-indexing
    #print("Total tardiness:", total_tardiness)

    return final_sequence, total_tardiness



In [13]:
import numpy as np
import warnings
def edd_sort(p, d):
    sorted_indices = np.lexsort((d, p))
    p = p[sorted_indices]
    d = d[sorted_indices]
    return p, d

def schedule_it(U, S, job):
    if job in U:
        index_to_delete = np.where(U == job)[0]
        U = np.delete(U, index_to_delete)
        S = np.append(S, job)
    else: 
        warnings.warn("logical error")
    return U, S

def calculate_tardiness(schedule, processing_times, due_dates):
    current_time = 0
    tardiness = 0
    for job in schedule:
        current_time += processing_times[job]
        tardiness += max(0, current_time - due_dates[job])
    return tardiness

def final_tardiness(p,d,seq):
    p = np.array(p)
    d = np.array(d)

    tardiness = np.cumsum(p[seq])-d[seq]
    tardiness = tardiness[tardiness>0]
    
    return tardiness.sum()

def PSK(p, d):
    p, d= edd_sort(p, d)
    U = np.arange(len(d))# Unscheduled jobs-indices after sorting
    S = np.array([])  # Scheduled jobs
    C = 0  # Completion time
    while len(U) > 0:
        job_i = U[0]
        i_ind = 0
        #job_i_ind = 0 #first job in U
        #two straightforward assignment steps
        if len(U) == 1: #step 1
            U, S = schedule_it(U, S, job_i) #schedule i
            break #end scheduling
        if C + p[job_i] >= d[job_i]: #step 2
            U, S = schedule_it(U, S, job_i)
            C += p[job_i]
            continue #continue the scheduling

        for j in range(i_ind+1, len(U)): #select j -next job in U
            #job_j = np.where(U == job_i)[0] + 1 #step 3
            job_j = U[j]

            if d[job_i]<= C + p[job_j]: #step 4
                U, S = schedule_it(U, S, job_i)
                C += p[job_i]
                break

            if d[job_i] <= d[job_j]: #step 5
                if U[-1] == job_j: #is j last job in U?
                    U, S = schedule_it(U, S, job_i)
                    C += p[job_i]
                    break
                else: # it is not
                    continue #select new j
            else:
                #step 6
                job_i = job_j # job_j becomes the active job now 
                if U[-1] == job_i:
                    C += p[job_i]
                    U, S = schedule_it(U, S, job_i)
                    break
                else:
                    continue
    #step 9
    S = S.astype(int) 
    tardiness = calculate_tardiness(S, p, d)

    return S, tardiness

In [14]:
opt_soln = [] 
tardiness_edd = []
tardiness_challenger = []
tardiness_mdd_chal = []
tardiness_mdd = []
tardiness_panners = []
tardiness_psk = []
for i in range(len(data)):
    o = data[i]['optimal_solution']
    opt_soln.append(o)
    p = data[i]["processing_times"].copy()
    d = data[i]["due_dates"].copy()
    
    #function call
    edd_schedule = EDD(p, d)
    mddchallenger_schedule = MDD_Challenger(p, d)
    challenger4_schedule = EDD_Challenger(p, d)
    mdd_schedule = MDD(p, d)
    panner_schedule, tt = pannerselvam(p, d)
    psk_schedule, tardiness__ = PSK(p, d)
    
    #tardiness
    temp_edd = final_tardiness(p,d,edd_schedule)
    temp_mdd_chal = final_tardiness(p, d, mddchallenger_schedule)
    temp_chal = final_tardiness(p, d, challenger4_schedule)
    temp_mdd = final_tardiness(p, d, mdd_schedule)
    temp_panner = final_tardiness(p, d, panner_schedule)
    
    #append to list
    tardiness_edd.append(temp_edd)
    tardiness_mdd_chal.append(temp_mdd_chal)
    tardiness_challenger.append(temp_chal)
    tardiness_mdd.append(temp_mdd)
    #tardiness_panners.append(tt)
    tardiness_panners.append(temp_panner)
    tardiness_psk.append(tardiness__)

[55 66 30 37 18 93 86 16 33 76 45 34 36 92 59 40 38 88 29 65 22 39  7 97
 74 47 32 54 90 43 23 46  4 52 84  9  1 79 28 57 61 27 56  8  6 15 94 83
 13 96  2 69 67 91 31 50 49 12 14 20 68 26 82 10 98  0 80 71 89 62 63  5
 53 95 21 81 85 78  3 11 73 77 60 58 44 99 42 72 19 41 17 24 35 70 51 48
 87 64 25 75]
[95 82 10  6 30 44 90 38 85 73  0 35 64 39 63 22 62 12 34 56 19 41 11 45
 69 77 78 80 36 72  3 53 66 16 68 89 43 93 97 79 21 49 13 86 71 84  1  9
  4 29 18 25 50 87 96 32 67 33 52  2 26 94 98 31 75 24 58 28 91 48 27  7
  5 81 65 37 23 88 47 74 60 55 54 76 61 99 92 83  8 20 46 59 40 70 57 42
 17 15 51 14]
[38 67 11 83 80 40 22 68 13 31 35 15 74 97 42 64 90 30 12 20 60 61 66  6
 71  3 23 52 91  0 48 55 73 79 85 63 95 27 51 59 58 57 81 86 82 87  2 14
 19 36 16 21 17  8 43 54  7 88 47 96 33 37  4 94 77 32 53 72 25 89 41 39
 29 99 65 18 46 45 44 70 34 62 56 50 69 78 28 75  9 92 24 76  1 98  5 49
 10 84 26 93]
[48 76 54 46 43 12 93 32 49  0 89 21 23 29  5 66 90 67  7 99 51 82 75 98
 53 73 70

In [15]:
chal_optgap = (sum(opt_soln)- sum(tardiness_challenger))/sum(opt_soln)
mddchal_optgap = (sum(opt_soln)- sum(tardiness_mdd_chal))/sum(opt_soln)
edd_optgap = (sum(opt_soln)- sum(tardiness_edd))/sum(opt_soln)
mdd_optgap = (sum(opt_soln)- sum(tardiness_mdd))/sum(opt_soln)
psk_optgap = (sum(opt_soln)- sum(tardiness_psk))/sum(opt_soln)
panner_optgap = (sum(opt_soln)- sum(tardiness_panners))/sum(opt_soln)
print('edd optimality gap: {}'.format(-edd_optgap))
print('edd challenger optimality gap: {}'.format(-chal_optgap))
print('mdd optimality gap: {}'.format(-mdd_optgap))
print('mdd challenger optimality gap: {}'.format(-mddchal_optgap))
print('psk optimality gap: {}'.format(-psk_optgap))
print('pannerselvam optimality gap: {}'.format(-panner_optgap))

print('optimal value: {}'.format(sum(opt_soln)/800))

edd optimality gap: 0.6611468488964342
edd challenger optimality gap: 0.056428387709533026
mdd optimality gap: 0.013341954709084928
mdd challenger optimality gap: 0.012080222125832756
psk optimality gap: 0.013168775727069924
pannerselvam optimality gap: 0.4405418584432366
optimal value: 8538.85375


In [16]:
#R = 0.2 T = 0.2
opt_soln = [] 
tardiness_edd = []
tardiness_challenger = []
tardiness_mdd_chal = []
tardiness_mdd = []
tardiness_panners = []
tardiness_psk = []
for i in range(0, 40):
#for i in range(0, 10):
    o = data[i]['optimal_solution']
    opt_soln.append(o)
    p = data[i]["processing_times"].copy()
    d = data[i]["due_dates"].copy()
    
    #function call
    edd_schedule = EDD(p, d)
    mddchallenger_schedule = MDD_Challenger(p, d)
    challenger4_schedule = EDD_Challenger(p, d)
    mdd_schedule = MDD(p, d)
    panner_schedule, tt = pannerselvam(p, d)
    psk_schedule, tardiness__ = PSK(p, d)
    
    #tardiness
    temp_edd = final_tardiness(p,d,edd_schedule)
    temp_mdd_chal = final_tardiness(p, d, mddchallenger_schedule)
    temp_chal = final_tardiness(p, d, challenger4_schedule)
    temp_mdd = final_tardiness(p, d, mdd_schedule)
    temp_panner = final_tardiness(p, d, panner_schedule)
    
    #append to list
    tardiness_edd.append(temp_edd)
    tardiness_mdd_chal.append(temp_mdd_chal)
    tardiness_challenger.append(temp_chal)
    tardiness_mdd.append(temp_mdd)
    #tardiness_panners.append(tt)
    tardiness_panners.append(temp_panner)
    tardiness_psk.append(tardiness__)
chal_optgap = (sum(opt_soln)- sum(tardiness_challenger))/sum(opt_soln)
mddchal_optgap = (sum(opt_soln)- sum(tardiness_mdd_chal))/sum(opt_soln)
edd_optgap = (sum(opt_soln)- sum(tardiness_edd))/sum(opt_soln)
mdd_optgap = (sum(opt_soln)- sum(tardiness_mdd))/sum(opt_soln)
psk_optgap = (sum(opt_soln)- sum(tardiness_psk))/sum(opt_soln)
panner_optgap = (sum(opt_soln)- sum(tardiness_panners))/sum(opt_soln)
print('edd optimality gap for 0.2, 0.2: {}'.format(-edd_optgap))
print('edd challenger optimality gap for 0.2, 0.2: {}'.format(-chal_optgap))
print('mdd optimality gap for 0.2, 0.2: {}'.format(-mdd_optgap))
print('mdd challenger optimality gap for 0.2, 0.2: {}'.format(-mddchal_optgap))
print('psk optimality gap for 0.2, 0.2: {}'.format(-psk_optgap))
print('pannerselvam optimality gap for 0.2, 0.2: {}'.format(-panner_optgap))
print('optimal value for 0.2, 0.2: {}'.format(sum(opt_soln)/40)) # 10 for 200-instance datasets, 40 for 800-instance datasets

[55 66 30 37 18 93 86 16 33 76 45 34 36 92 59 40 38 88 29 65 22 39  7 97
 74 47 32 54 90 43 23 46  4 52 84  9  1 79 28 57 61 27 56  8  6 15 94 83
 13 96  2 69 67 91 31 50 49 12 14 20 68 26 82 10 98  0 80 71 89 62 63  5
 53 95 21 81 85 78  3 11 73 77 60 58 44 99 42 72 19 41 17 24 35 70 51 48
 87 64 25 75]
[95 82 10  6 30 44 90 38 85 73  0 35 64 39 63 22 62 12 34 56 19 41 11 45
 69 77 78 80 36 72  3 53 66 16 68 89 43 93 97 79 21 49 13 86 71 84  1  9
  4 29 18 25 50 87 96 32 67 33 52  2 26 94 98 31 75 24 58 28 91 48 27  7
  5 81 65 37 23 88 47 74 60 55 54 76 61 99 92 83  8 20 46 59 40 70 57 42
 17 15 51 14]
[38 67 11 83 80 40 22 68 13 31 35 15 74 97 42 64 90 30 12 20 60 61 66  6
 71  3 23 52 91  0 48 55 73 79 85 63 95 27 51 59 58 57 81 86 82 87  2 14
 19 36 16 21 17  8 43 54  7 88 47 96 33 37  4 94 77 32 53 72 25 89 41 39
 29 99 65 18 46 45 44 70 34 62 56 50 69 78 28 75  9 92 24 76  1 98  5 49
 10 84 26 93]
[48 76 54 46 43 12 93 32 49  0 89 21 23 29  5 66 90 67  7 99 51 82 75 98
 53 73 70

In [12]:
#R = 0.2 T = 0.6
opt_soln = [] 
tardiness_edd = []
tardiness_challenger = []
tardiness_mdd_chal = []
tardiness_mdd = []
tardiness_panners = []
tardiness_psk = []
for i in range(80,120):
#for i in range(20, 30):
    o = data[i]['optimal_solution']
    opt_soln.append(o)
    p = data[i]["processing_times"].copy()
    d = data[i]["due_dates"].copy()
    
    #function call
    
    edd_schedule = EDD(p, d)
    mddchallenger_schedule = MDD_Challenger(p, d)
    challenger4_schedule = EDD_Challenger(p, d)
    mdd_schedule = MDD(p, d)
    panner_schedule, tt = pannerselvam(p, d)
    psk_schedule, tardiness__ = PSK(p, d)
    
    #tardiness
    temp_edd = final_tardiness(p,d,edd_schedule)
    temp_mdd_chal = final_tardiness(p, d, mddchallenger_schedule)
    temp_chal = final_tardiness(p, d, challenger4_schedule)
    temp_mdd = final_tardiness(p, d, mdd_schedule)
    temp_panner = final_tardiness(p, d, panner_schedule)
    
    #append to list
    tardiness_edd.append(temp_edd)
    tardiness_mdd_chal.append(temp_mdd_chal)
    tardiness_challenger.append(temp_chal)
    tardiness_mdd.append(temp_mdd)
    #tardiness_panners.append(tt)
    tardiness_panners.append(temp_panner)
    tardiness_psk.append(tardiness__)
chal_optgap = (sum(opt_soln)- sum(tardiness_challenger))/sum(opt_soln)
mddchal_optgap = (sum(opt_soln)- sum(tardiness_mdd_chal))/sum(opt_soln)
edd_optgap = (sum(opt_soln)- sum(tardiness_edd))/sum(opt_soln)
mdd_optgap = (sum(opt_soln)- sum(tardiness_mdd))/sum(opt_soln)
psk_optgap = (sum(opt_soln)- sum(tardiness_psk))/sum(opt_soln)
panner_optgap = (sum(opt_soln)- sum(tardiness_panners))/sum(opt_soln)

print('edd optimality gap for 0.2, 0.6: {}'.format(-edd_optgap))
print('edd challenger optimality gap for 0.2, 0.6: {}'.format(-chal_optgap))
print('mdd optimality gap for 0.2, 0.6: {}'.format(-mdd_optgap))
print('mdd challenger optimality gap for 0.2, 0.6: {}'.format(-mddchal_optgap))
print('psk optimality gap for 0.2, 0.6: {}'.format(-psk_optgap))
print('pannerselvam optimality gap for 0.2, 0.6: {}'.format(-panner_optgap))

print('optimal value for 0.2, 0.6: {}'.format(sum(opt_soln)/40))

edd optimality gap for 0.2, 0.6: 0.36000563674494557
edd challenger optimality gap for 0.2, 0.6: 0.041297446057179804
mdd optimality gap for 0.2, 0.6: 0.058903984681316676
mdd challenger optimality gap for 0.2, 0.6: 0.029286205724611852
psk optimality gap for 0.2, 0.6: 0.05778492502300289
pannerselvam optimality gap for 0.2, 0.6: 0.22475691537422185
optimal value for 0.2, 0.6: 3015.925


In [13]:
#R = 0.2 T = 0.8
opt_soln = [] 
tardiness_edd = []
tardiness_challenger = []
tardiness_mdd_chal = []
tardiness_mdd = []
tardiness_panners = []
tardiness_psk = []
#for i in range(30, 40):
for i in range(120,160):

    o = data[i]['optimal_solution']
    opt_soln.append(o)
    p = data[i]["processing_times"].copy()
    d = data[i]["due_dates"].copy()
    
    #function call
    edd_schedule = EDD(p, d)
    mddchallenger_schedule = MDD_Challenger(p, d)
    challenger4_schedule = EDD_Challenger(p, d)
    mdd_schedule = MDD(p, d)
    panner_schedule, tt = pannerselvam(p, d)
    psk_schedule, tardiness__ = PSK(p, d)
    
    #tardiness
    temp_edd = final_tardiness(p,d,edd_schedule)
    temp_mdd_chal = final_tardiness(p, d, mddchallenger_schedule)
    temp_chal = final_tardiness(p, d, challenger4_schedule)
    temp_mdd = final_tardiness(p, d, mdd_schedule)
    temp_panner = final_tardiness(p, d, panner_schedule)
    
    #append to list
    tardiness_edd.append(temp_edd)
    tardiness_mdd_chal.append(temp_mdd_chal)
    tardiness_challenger.append(temp_chal)
    tardiness_mdd.append(temp_mdd)
    tardiness_panners.append(temp_panner)
    tardiness_psk.append(tardiness__)
chal_optgap = (sum(opt_soln)- sum(tardiness_challenger))/sum(opt_soln)
mddchal_optgap = (sum(opt_soln)- sum(tardiness_mdd_chal))/sum(opt_soln)
edd_optgap = (sum(opt_soln)- sum(tardiness_edd))/sum(opt_soln)
mdd_optgap = (sum(opt_soln)- sum(tardiness_mdd))/sum(opt_soln)
psk_optgap = (sum(opt_soln)- sum(tardiness_psk))/sum(opt_soln)
panner_optgap = (sum(opt_soln)- sum(tardiness_panners))/sum(opt_soln)
print('edd optimality gap for 0.2, 0.8: {}'.format(-edd_optgap))
print('edd challenger optimality gap for 0.2, 0.8: {}'.format(-chal_optgap))
print('mdd optimality gap for 0.2, 0.8: {}'.format(-mdd_optgap))
print('mdd challenger optimality gap for 0.2, 0.8: {}'.format(-mddchal_optgap))
print('psk optimality gap for 0.2, 0.8: {}'.format(-psk_optgap))
print('pannerselvam optimality gap for 0.2, 0.8: {}'.format(-panner_optgap))

print('optimal value for 0.2, 0.8: {}'.format(sum(opt_soln)/40))

edd optimality gap for 0.2, 0.8: 0.3641995435278774
edd challenger optimality gap for 0.2, 0.8: 0.014811433539832627
mdd optimality gap for 0.2, 0.8: 0.02251929138137159
mdd challenger optimality gap for 0.2, 0.8: 0.007086186284099554
psk optimality gap for 0.2, 0.8: 0.02251929138137159
pannerselvam optimality gap for 0.2, 0.8: 0.113718074122378
optimal value for 0.2, 0.8: 5750.625


/var/folders/jk/gtxpwc995ddbcqbt31_0qkgr0000gn/T/ipykernel_11990/4279036889.py:17: RuntimeWarning: divide by zero encountered in scalar divide
  USi.append((Si / processing_times[i], i))


In [16]:
#R = 0.6 T = 0.4
opt_soln = [] 
tardiness_edd = []
tardiness_challenger = []
tardiness_mdd_chal = []
tardiness_mdd = []
tardiness_panners = []
tardiness_psk = []
for i in range(360, 400):
#for i in range(90, 100):

    o = data[i]['optimal_solution']
    opt_soln.append(o)
    p = data[i]["processing_times"].copy()
    d = data[i]["due_dates"].copy()
    
    #function call
    edd_schedule = EDD(p, d)
    mddchallenger_schedule = MDD_Challenger(p, d)
    challenger4_schedule = EDD_Challenger(p, d)
    mdd_schedule = MDD(p, d)
    panner_schedule, tt = pannerselvam(p, d)
    psk_schedule, tardiness__ = PSK(p, d)
    
    #tardiness
    temp_edd = final_tardiness(p,d,edd_schedule)
    temp_mdd_chal = final_tardiness(p, d, mddchallenger_schedule)
    temp_chal = final_tardiness(p, d, challenger4_schedule)
    temp_mdd = final_tardiness(p, d, mdd_schedule)
    temp_panner = final_tardiness(p, d, panner_schedule)
    
    #append to list
    tardiness_edd.append(temp_edd)
    tardiness_mdd_chal.append(temp_mdd_chal)
    tardiness_challenger.append(temp_chal)
    tardiness_mdd.append(temp_mdd)
    tardiness_panners.append(temp_panner)
    tardiness_psk.append(tardiness__)
chal_optgap = (sum(opt_soln)- sum(tardiness_challenger))/sum(opt_soln)
mddchal_optgap = (sum(opt_soln)- sum(tardiness_mdd_chal))/sum(opt_soln)
edd_optgap = (sum(opt_soln)- sum(tardiness_edd))/sum(opt_soln)
mdd_optgap = (sum(opt_soln)- sum(tardiness_mdd))/sum(opt_soln)
psk_optgap = (sum(opt_soln)- sum(tardiness_psk))/sum(opt_soln)
panner_optgap = (sum(opt_soln)- sum(tardiness_panners))/sum(opt_soln)
print('edd optimality gap for 0.6, 0.4: {}'.format(-edd_optgap))
print('edd challenger optimality gap for 0.6, 0.4: {}'.format(-chal_optgap))
print('mdd optimality gap for 0.6, 0.4: {}'.format(-mdd_optgap))
print('mdd challenger optimality gap for 0.6, 0.4: {}'.format(-mddchal_optgap))
print('psk optimality gap for 0.6, 0.4: {}'.format(-psk_optgap))
print('pannerselvam optimality gap for 0.6, 0.4: {}'.format(-panner_optgap))

print('optimal value for 0.6, 0.4: {}'.format(sum(opt_soln)/40))

edd optimality gap for 0.6, 0.4: 0.29873787420465214
edd challenger optimality gap for 0.6, 0.4: 0.3225200792740169
mdd optimality gap for 0.6, 0.4: 0.05152811098362366
mdd challenger optimality gap for 0.6, 0.4: 0.042244706373213724
psk optimality gap for 0.6, 0.4: 0.05152811098362366
pannerselvam optimality gap for 0.6, 0.4: 1.3020757275477208
optimal value for 0.6, 0.4: 479.35


/var/folders/jk/gtxpwc995ddbcqbt31_0qkgr0000gn/T/ipykernel_11990/4279036889.py:17: RuntimeWarning: divide by zero encountered in scalar divide
  USi.append((Si / processing_times[i], i))


In [17]:
#R = 0.8 T = 0.8
opt_soln = [] 
tardiness_edd = []
tardiness_challenger = []
tardiness_mdd_chal = []
tardiness_mdd = []
tardiness_panners = []
tardiness_psk = []
#for i in range(150, 160):
for i in range(600, 640):
    o = data[i]['optimal_solution']
    opt_soln.append(o)
    p = data[i]["processing_times"].copy()
    d = data[i]["due_dates"].copy()
    
    #function call
    edd_schedule = EDD(p, d)
    mddchallenger_schedule = MDD_Challenger(p, d)
    challenger4_schedule = EDD_Challenger(p, d)
    mdd_schedule = MDD(p, d)
    panner_schedule, tt = pannerselvam(p, d)
    psk_schedule, tardiness__ = PSK(p, d)
    
    #tardiness
    temp_edd = final_tardiness(p,d,edd_schedule)
    temp_mdd_chal = final_tardiness(p, d, mddchallenger_schedule)
    temp_chal = final_tardiness(p, d, challenger4_schedule)
    temp_mdd = final_tardiness(p, d, mdd_schedule)
    temp_panner = final_tardiness(p, d, panner_schedule)
    
    #append to list
    tardiness_edd.append(temp_edd)
    tardiness_mdd_chal.append(temp_mdd_chal)
    tardiness_challenger.append(temp_chal)
    tardiness_mdd.append(temp_mdd)
    tardiness_panners.append(temp_panner)
    tardiness_psk.append(tardiness__)
chal_optgap = (sum(opt_soln)- sum(tardiness_challenger))/sum(opt_soln)
mddchal_optgap = (sum(opt_soln)- sum(tardiness_mdd_chal))/sum(opt_soln)
edd_optgap = (sum(opt_soln)- sum(tardiness_edd))/sum(opt_soln)
mdd_optgap = (sum(opt_soln)- sum(tardiness_mdd))/sum(opt_soln)
psk_optgap = (sum(opt_soln)- sum(tardiness_psk))/sum(opt_soln)
panner_optgap = (sum(opt_soln)- sum(tardiness_panners))/sum(opt_soln)
print('edd optimality gap for 0.8, 0.8: {}'.format(-edd_optgap))
print('edd challenger optimality gap for 0.8, 0.8: {}'.format(-chal_optgap))
print('mdd optimality gap for 0.8, 0.8: {}'.format(-mdd_optgap))
print('mdd challenger optimality gap for 0.8, 0.8: {}'.format(-mddchal_optgap))
print('psk optimality gap for 0.8, 0.8: {}'.format(-psk_optgap))
print('pannerselvam optimality gap for 0.8, 0.8: {}'.format(-panner_optgap))

print('optimal value for 0.8, 0.8: {}'.format(sum(opt_soln)/40))

edd optimality gap for 0.8, 0.8: 0.3469917080391412
edd challenger optimality gap for 0.8, 0.8: 0.012448857508394574
mdd optimality gap for 0.8, 0.8: 0.0021832440386521526
mdd challenger optimality gap for 0.8, 0.8: 0.0019692861228642416
psk optimality gap for 0.8, 0.8: 0.0021570451101883266
pannerselvam optimality gap for 0.8, 0.8: 0.23995161931210346
optimal value for 0.8, 0.8: 5725.425


/var/folders/jk/gtxpwc995ddbcqbt31_0qkgr0000gn/T/ipykernel_11990/4279036889.py:17: RuntimeWarning: divide by zero encountered in scalar divide
  USi.append((Si / processing_times[i], i))


In [17]:
################################################################
#Comparison of Local Search Variations

In [39]:
import numpy as np
import os
path = 'data/data_20_800_new_.npy'
path = 'data/data_100_200.npy'
path = 'data/data_200_200.npy'
path = 'data/data_500_200.npy'
with open(path, 'rb') as f:
    data = np.load(f, allow_pickle=True)
len(data)

200

In [29]:
def Augmented_MDD_Challenger(processing_times: np.ndarray, due_dates: np.ndarray) -> np.ndarray:
    """ Find mathematical heuristic function skeleton for the single machine scheduling problem. Each job is assigned to the machine exactly once. Do not manipulate original due dates or processing times.

    Args:
        processing_times: A numpy array representing processing times of jobs.
        due_dates: A numpy array representing due dates of jobs.

    Return:
        A numpy array representing indices of jobs assigned in order as the result of applying the mathematical function to the inputs.
    """
    """Ultimate version of the job assignment function for the single machine scheduling problem."""
    current_time = 0
    schedule = []
    unscheduled_jobs = np.arange(len(processing_times))
    weighting = 1.1
    
    sort_order = np.lexsort((processing_times[unscheduled_jobs], due_dates[unscheduled_jobs]))
    unscheduled_jobs = unscheduled_jobs[sort_order]
    
    while unscheduled_jobs.size > 0:
        urgencies = np.maximum(processing_times[unscheduled_jobs] * 1.1 + current_time, due_dates[unscheduled_jobs])
        urgency_multiplier = (processing_times[unscheduled_jobs] / (current_time + np.max(processing_times[unscheduled_jobs])))
        urgency_multiplier[urgency_multiplier > 1] = 1
        urgency_bonus = (urgency_multiplier ** 2) / (1 + urgency_multiplier ** 2)
        urgencies *= (1 + urgency_bonus)
        impact_on_current_time = processing_times[unscheduled_jobs] / (current_time + np.sum(processing_times[unscheduled_jobs]) / len(unscheduled_jobs))
        urgencies += impact_on_current_time
        next_job = unscheduled_jobs[np.argmin(urgencies)]
        schedule.append(next_job)
        current_time += processing_times[next_job]
        unscheduled_jobs = np.delete(unscheduled_jobs, np.where(unscheduled_jobs == next_job))

        #search
        best_tardiness = calculate_tardiness(schedule, p, d)
        best_schedule = schedule
        for i in range(len(schedule) - 1):
            trial_schedule = schedule[:i] + schedule[i+1:] + [schedule[i]]
            trial_tardiness = calculate_tardiness(trial_schedule, p, d)
            if trial_tardiness < best_tardiness:
                best_tardiness = trial_tardiness
                best_schedule = trial_schedule[:]
        schedule = best_schedule
    
    return np.array(schedule)

In [19]:
def AUG_MDD(p, d):
    # Initialize current time
    current_time = 0

    # Create an array to track the scheduling order
    schedule = []

    # Number of jobs
    num_jobs = len(p)

    # Create an index array for tracking unscheduled jobs
    unscheduled_jobs = np.arange(num_jobs)

    # While there are unscheduled jobs
    while unscheduled_jobs.size > 0:
        # Calculate urgencies for each unscheduled job
        #urgencies = np.maximum(p[unscheduled_jobs], d[unscheduled_jobs] - current_time)
        urgencies = np.maximum(p[unscheduled_jobs]+ current_time, d[unscheduled_jobs] )
        # Find the job with the minimum urgency
        index_min_urgency = np.argmin(urgencies)
        job_to_schedule = unscheduled_jobs[index_min_urgency]
        
        # Append job id (index + 1 for 1-based id) to schedule
        schedule.append(job_to_schedule)
        
        # Update current time
        current_time += p[job_to_schedule]
        
        # Remove the job from the list of unscheduled jobs
        unscheduled_jobs = np.delete(unscheduled_jobs, index_min_urgency)

        best_tardiness = calculate_tardiness(schedule, p, d)
        best_schedule = schedule[:]
        for i in range(len(schedule) - 1):
            trial_schedule = schedule[:i] + schedule[i+1:] + [schedule[i]]
            trial_tardiness = calculate_tardiness(trial_schedule, p, d)
            if trial_tardiness < best_tardiness:
                best_tardiness = trial_tardiness
                best_schedule = trial_schedule[:]
        schedule = best_schedule
    return schedule

In [40]:
opt_soln = [] 
tardiness_mdd = []
tardiness_mdd_chal = []
tardiness_mddaug = []
tardiness_mddcaug = []


for i in range(len(data)):
    o = data[i]['optimal_solution']
    opt_soln.append(o)
    p = data[i]["processing_times"].copy()
    d = data[i]["due_dates"].copy()
    
    #function call
    mdd_schedule = MDD(p, d)
    mddchallenger_schedule = MDD_Challenger(p, d)
    augmddc_schedule = Augmented_MDD_Challenger(p, d)
    augmdd_schedule = AUG_MDD(p, d)
    
    #tardiness
    temp_mdd = final_tardiness(p, d, mdd_schedule)
    temp_mdd_chal = final_tardiness(p, d, mddchallenger_schedule)
    temp_augmddc = final_tardiness(p, d, augmddc_schedule)
    temp_augmdd = final_tardiness(p, d, augmdd_schedule)
    
    #append to list
    tardiness_mdd.append(temp_mdd)
    tardiness_mdd_chal.append(temp_mdd_chal)
    tardiness_mddcaug.append(temp_augmddc)
    tardiness_mddaug.append(temp_augmdd)


tardiness_mdd = np.array(tardiness_mdd)
tardiness_mdd_chal = np.array(tardiness_mdd_chal)
tardiness_mddcaug = np.array(tardiness_mddcaug)
tardiness_mddaug = np.array(tardiness_mddaug)

mdd_optgap = (sum(opt_soln)- sum(tardiness_mdd))/sum(opt_soln)
mddchal_optgap = (sum(opt_soln)- sum(tardiness_mdd_chal))/sum(opt_soln)
mddcaug_optgap = (sum(opt_soln)- sum(tardiness_mddcaug))/sum(opt_soln)
mddaug_optgap = (sum(opt_soln)- sum(tardiness_mddaug))/sum(opt_soln)

print('average mdd optimality gap: {}'.format(-mdd_optgap))
print('average augmented mdd optimality gap: {}'.format(-mddaug_optgap))
print('average mdd challenger optimality gap: {}'.format(-mddchal_optgap))
print('average aug mdd challenger optimality gap: {}'.format(-mddcaug_optgap))

average mdd optimality gap: 0.013354593754150063
average augmented mdd optimality gap: 0.010723505592516626
average mdd challenger optimality gap: 0.012927649691082002
average aug mdd challenger optimality gap: 0.010400324832961733


In [41]:
# testing

# Function to calculate averages for specified ranges, considering only nonzero optimal solutions
def calculate_averages(percent_deviation, opt_soln, ranges):
    averages = {}
    for start, end in ranges:
        # Filter out zero optimal solutions within the range
        nonzero_indices_in_range = np.where(opt_soln[start:end] != 0)[0] + start
        if len(nonzero_indices_in_range) > 0:
            averages[(start, end)] = np.mean(percent_deviation[nonzero_indices_in_range])
        else:
            averages[(start, end)] = None  # No nonzero optimal solutions in this range
    return averages

def calculate_zero_optimal_deviation(opt_soln, tardiness):
    zero_indices = np.where(opt_soln == 0)
    total_deviation = np.sum(tardiness[zero_indices])
    zero_count = len(zero_indices[0])  # Access the first element as `where` returns a tuple
    return total_deviation, zero_count


def results(opt_soln, tardiness, ranges):
    # Calculate averages
    percent_deviation = np.zeros_like(opt_soln, dtype=float)
    nonzero_indices = np.where(opt_soln != 0)
    percent_deviation[nonzero_indices] = ((tardiness[nonzero_indices] - opt_soln[nonzero_indices]) / opt_soln[nonzero_indices]) * 100
    # percent deviation calculation for nonzero optimal cases
    averages = calculate_averages(percent_deviation, opt_soln, ranges)

    # Results
    print("Averages for Specified Ranges:", averages)

    # Perform calculations
    zero_deviation, zero_count = calculate_zero_optimal_deviation(opt_soln, tardiness)

    # Results
    print("Total Deviation for Zero Optimal Solutions:", zero_deviation)
    print("Number of Zero Optimal Solutions:", zero_count)

    return averages, zero_deviation, zero_count

In [42]:
import pandas as pd
opt_soln = np.array(opt_soln)
def display_results_table(opt_soln, tardiness_list, ranges, algorithm_names):
    results_data = []
    for tardiness, algo_name in zip(tardiness_list, algorithm_names):
        # Calculate results using the provided results function
        averages, zero_deviation, zero_count = results(opt_soln, tardiness, ranges)
        
        # Prepare data for each algorithm and range
        for range_key, avg in averages.items():
            results_data.append({
                'Algorithm': algo_name,
                'Range': f'{range_key[0]}-{range_key[1]}',
                'Average Deviation': avg,
                'Total Zero Deviation': zero_deviation,
                'Zero Count': zero_count
            })
    
    # Create a DataFrame to display results
    results_df = pd.DataFrame(results_data)
    return results_df

tardiness_list = [tardiness_mdd, tardiness_mddaug, tardiness_mdd_chal, tardiness_mddcaug]
algorithm_names = ["MDD", "Augmented MDD", "MDDC",  "Augmented MDDC"]
ranges = [(0,200)] # for 100, 200 and 500-job datasets
results_df = display_results_table(opt_soln, tardiness_list, ranges, algorithm_names)

tardiness_mdd = np.array(tardiness_mdd)
tardiness_mdd_chal = np.array(tardiness_mdd_chal)
tardiness_mddcaug = np.array(tardiness_mddcaug)
tardiness_mddaug = np.array(tardiness_mddaug)

Averages for Specified Ranges: {(0, 200): 48.74581421104253}
Total Deviation for Zero Optimal Solutions: 0.0
Number of Zero Optimal Solutions: 40
Averages for Specified Ranges: {(0, 200): 48.14724232877593}
Total Deviation for Zero Optimal Solutions: 0.0
Number of Zero Optimal Solutions: 40
Averages for Specified Ranges: {(0, 200): 48.39517719340983}
Total Deviation for Zero Optimal Solutions: 0.0
Number of Zero Optimal Solutions: 40
Averages for Specified Ranges: {(0, 200): 47.696919050964866}
Total Deviation for Zero Optimal Solutions: 0.0
Number of Zero Optimal Solutions: 40


In [43]:
pivot_avg_dev = results_df.pivot(index='Range', columns='Algorithm', values='Average Deviation')
pivot_avg_dev.sort_index()
# Pivot table for Total Zero Deviation
pivot_zero_dev = results_df.pivot(index='Range', columns='Algorithm', values='Total Zero Deviation')

# Display results
print("Pivot Table - Average Deviation:")
pivot_avg_dev


Pivot Table - Average Deviation:


Algorithm,Augmented MDD,Augmented MDDC,MDD,MDDC
Range,,,,
0-200,48.147242,47.696919,48.745814,48.395177
